# Analyse des vergangenen Sommers

Dieses Notebook kann lokal oder **direkt im Browser** auf [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/meteotest/urban-heat-API-docs/add-heatdays-and-tropicalnights-analysis?labpath=python_data_analysis.ipynb) oder [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meteotest/urban-heat-API-docs/blob/add-heatdays-and-tropicalnights-analysis/python_data_analysis.ipynb) ausgeführt werden.

In [1]:
import requests 
import pandas as pd
import geopandas as gpd 
import matplotlib # for plotting
import folium
import branca

In [2]:
def get_stations() -> gpd.GeoDataFrame:
    response = requests.get(
            url=f"https://smart-urban-heat-map.ch/api/v2/latest"
    )
    stations = gpd.read_file(response.text)
    stations.drop(["temperature", "relativeHumidity", "dateObserved"], axis=1, inplace=True)
    
    return stations

def get_station_analysis(time_from: str, time_to: str) -> pd.DataFrame:
    stations = get_stations()
    
    # TO DO: REMOVE SUBSETTING OF STATIONS TO RUN OVER ALL STATIONS
    # subset of stations for testing
    station_name_subset = ['Gurten Kulm', 'Breitenrain, Waffenweg'] 
    stations = stations.loc[stations['name'].isin(station_name_subset)] 

    stations['einschlaftemperatur'] = None
    
    for idx, station in stations.iterrows():
        station_id = station.stationId
        response = requests.get(url=f"https://smart-urban-heat-map.ch/api/v2/timeseries?stationId={station_id}&timeFrom={time_from}&timeTo={time_to}")
        
        payload = response.json()
        if payload["values"] == []:
            continue
            
        df = pd.DataFrame(payload["values"])

        df["dateObserved"] = pd.to_datetime(df["dateObserved"])
        df["dateObserved"] = df["dateObserved"].dt.tz_convert("Europe/Zurich")
        
        # hier werden die Hitzetage und Tropennächte berechnet
        einschlaftemperatur = calc_einschlaftemperatur(df)
        
        stations.loc[idx, 'einschlaftemperatur'] = einschlaftemperatur

    return stations

def calc_einschlaftemperatur(df: pd.DataFrame) -> float:
    sleep_time_mask = (df['dateObserved'].dt.hour >= 22) & (df['dateObserved'].dt.hour < 23)
    sleep_time = df.loc[sleep_time_mask]

    nightly_sleep_temperatures = (
        sleep_time.groupby(sleep_time['dateObserved'].dt.date)['temperature'].mean()
    )

    return nightly_sleep_temperatures.mean()


# TO DO: Calculate UHI for selected stations (make the stations definable by user)
# compared to 'Zollikofen 3m' as the rural reference station
# - make sure zollikofen data is always downloaded even if it is inactive
# get id and name of zollikofen from here:
# https://smart-urban-heat-map.ch/api/v2/stations
#         "name": "Zollikofen 3m",
#         "stationId": "11119",
#         "latestMeasurementDate": "2024-07-16T11:00:38Z"


# TO DO: UHI function
# def calc_uhi(df: pd.DataFrame) -> float:
#     # dummy definition of UHI
#      uhi = float(2.0)
#      return uhi

In [3]:
stations = get_stations()
stations

,stationId,name,geometry
0,11002,Bundesplatz,POINT (7.44353 46.94692)
1,11023,"Obstberg, Wattenwylweg 32",POINT (7.46407 46.94714)
2,11087,Oberwangen Laterne,POINT (7.36066 46.91679)
3,11004,Schosshaldenfriedhof 2,POINT (7.47186 46.95339)
4,11024,Wankdorf ESP,POINT (7.46244 46.96798)
...,...,...,...
94,11025,Elfenau,POINT (7.4695 46.93558)
95,11089,Niederwangen Zentrum Laterne,POINT (7.37709 46.92567)
96,11069,Wankdorf ESP 2,POINT (7.46247 46.968)
97,12010,Reitplatz Grünau,POINT (7.30167 47.07714)


In [5]:
# get data of specific period (i.e. a heatwave)
time_from = "2024-06-01T22:00:00Z"
time_to = "2024-06-30T23:50:00Z"


station_analysis = get_station_analysis(time_from, time_to)
station_analysis

,stationId,name,geometry,einschlaftemperatur
21,11003,"Breitenrain, Waffenweg",POINT (7.45192 46.96173),18.458032
65,11075,Gurten Kulm,POINT (7.43971 46.91833),15.826256


In [6]:
# Einschlaftemperatur Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

# TO DO: Add title to the map containing
# definition of einschlaftemperatur and date range

colormap = branca.colormap.linear.YlOrRd_09
einschlaftemperatur = station_analysis.einschlaftemperatur.values

# Define colourmap range depending on values of 'einschlaftemperatur
vmin = einschlaftemperatur.min()
vmax = einschlaftemperatur.max()

# Define the colormap with the specified range
colormap = branca.colormap.linear.YlOrRd_09.scale(vmin, vmax)

# Convert to step colormap with a specified number of steps
n_steps = 10  # Define the number of steps
colormap = colormap.to_step(n_steps)

# colormap = colormap.scale(0, einschlaftemperatur).to_step(einschlaftemperatur) 
colormap.caption = "Mittlere Einschlaftemperatur"
colormap.add_to(m)

# TO DO: Plot einschlaftemperature value directly, dont plot point (circle marker)
for idx, station in station_analysis.iterrows():
    folium.CircleMarker(
        location=(station.geometry.y, station.geometry.x),
        radius=5,
        color="black",
        weight=0.5,
        fill=True,
        fill_color=colormap(station.einschlaftemperatur),
        fill_opacity=1,
        popup=f"Einschlaftemperatur {station.einschlaftemperatur:.1f} °C",
        tooltip=f"{station['name']}: Einschlaftemperatur {station.einschlaftemperatur:.1f} °C",
    ).add_to(m)

m

In [8]:
# "Urban Heat Island" Effekt Grafik
